# Curso básico de Python aplicado à Astronomia
### Laboratório Interinstitucional de e-Astronomia


# Aula 3 - Acesso a dados pelo LIneA Science Server, Pandas, Astropy




Última atualização: 04/02/2021

### Objetivos

Ao final desta aula, os participantes saberão:

- Acessar o LIneA Science Server
- Fazer queries no banco de dados
- Fazer download dos dados 
- Ler arquivos a partir de um notebook (usando a instalação local)
- Criar um Dataframe e imprimir estatísticas básicas com a biblioteca Pandas 
- Converter unidades usando a biblioteca Astropy







### Índice
1. SQL básico ([slides aula 3](https://docs.google.com/presentation/d/1jo1YSXiC-lUm14zUaI-BMPXr9qfpoqKCnHQGO0i-OQI/edit?usp=sharing))
2. Queries e download dos dados no [LIneA Science Server](https://desportal2.cosmology.illinois.edu )
3. [Pandas DataFrame](#pandas)
4. [Astropy](#astropy)

# Dados


```sql

SELECT COUNT(*)
FROM DES_ADMIN.DR1_MAIN
WHERE ra > 35 and ra < 36
AND dec > -10 and dec < -9
AND mag_auto_g between 15 and 23
AND CLASS_STAR_G < 0.95
```


```sql

SELECT coadd_object_id, ra ,dec, mag_auto_g, mag_auto_r, mag_auto_i
FROM DES_ADMIN.DR1_MAIN
WHERE ra > 35 and ra < 36
AND dec > -10 and dec < -9
AND mag_auto_g between 15 and 23
AND CLASS_STAR_G < 0.95
```


<a class="anchor" id="imports"></a>
# 1. Importando bibliotecas

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from astropy.io import fits
from astropy.table import Table
import matplotlib.pyplot as plt

In [ ]:
print('Numpy version: ', np.__version__)
print('Pandas version: ', pd.__version__)
print('Seaborn version: ', sns.__version__)

<a class="anchor" id="read"></a>
# 2. Leitura e escrita de arquivos


Nos exemplos a seguir vamos explorar os dados de uma amostra pública de galáxias do levantamento 
[Canada-France-Hawaii Telescope Legacy Survey (CFHTLS)](https://www.cfht.hawaii.edu/Science/CFHLS/). Esta amostra é um catálogo fotométrico, ou seja, uma tabela com medidas feitas a partir de fotografias do céu. Cada linha da tabela representa uma galáxia detectada nas imagens e cada coluna um atributo.   




Os dados estão armazenados em um arquivo no formato FITS, comum na astronomia. Para para ler o arquivo e extrair os dados, vamos precisar importar a biblioteca `astropy`. 

In [ ]:
hdulist = fits.open('/archive/user/julia/jupyter/data/cfhtls/cfhtw.fits')
catalog = hdulist[1].data 

In [ ]:
print('A amostra contém %d galáxias e %d atributos' %(len(catalog),  len(catalog.columns)))

In [ ]:
catalog.columns

Atribuindo as colunas de interesse a variáveis e selecionando os mil primeiros objetos

In [ ]:
ra = catalog['RA'][0:1000]
dec = catalog['DEC'][0:1000]
zphot = catalog['ZPHOT'][0:1000]
mag_abs_u = catalog['MAG_ABS_U'][0:1000]
mag_abs_r = catalog['MAG_ABS_R'][0:1000]

In [ ]:
type(ra)

In [ ]:
len(ra)

In [ ]:
ra[0:10]

In [ ]:
zphot[0:10]

In [ ]:
mag_abs_u[0:10]

Salvando a seleção em um arquivo de texto formato CSV

In [ ]:
cols_to_save = np.c_[ra, dec, zphot, mag_abs_u, mag_abs_r]
cols_format = ['%.4f', '%.4f', '%.4f', '%.3f', '%.3f']
text_header = 'ra,dec,zphot,mag_u,mag_r' # sem espaços

np.savetxt('dados/galaxias.csv', cols_to_save, fmt=cols_format, delimiter=',', header=text_header, comments='')

In [ ]:
# limpando a memória
del ra, dec, zphot, mag_abs_u, mag_abs_r, catalog

<a class="anchor" id="stats"></a>
# 3. Visualização de tabelas e estatísticas básicas

Criando um Pandas DataFrame a partir do arquivo no formato CSV.

In [ ]:
df = pd.read_csv('dados/galaxias.csv')

In [ ]:
df

In [ ]:
df.head() 

In [ ]:
df.head(2) 

In [ ]:
df.tail()

Estatísticas básicas

In [ ]:
df.describe()

<a class="anchor" id="plots"></a>
# 4. Exemplos de gráficos estáticos e interativos

### 4.1 Utilizando a biblioteca Matplotlib

In [ ]:
%matplotlib inline

In [ ]:
plt.scatter(df['ra'], df['dec'])

In [ ]:
def dist_esp():
    plt.figure(dpi=100)
    plt.scatter(df['ra'], df['dec'], c=df['zphot'], cmap='rainbow')
    plt.xlim(df['ra'].min()-0.02, df['ra'].max()+0.02)
    plt.ylim(df['dec'].min()-0.001, df['dec'].max()+0.001)
    cbar = plt.colorbar()
    cbar.set_label('redshift', size=10)
    plt.title('Distribuição Espacial')
    plt.xlabel('Ascensão Reta')
    plt.ylabel('Declinação')
    plt.xticks(size=8)
    plt.yticks(size=8)
    plt.tight_layout()

In [ ]:
dist_esp()

### 4.2 Utilizando a biblioteca Seaborn

In [ ]:
# Histograma simples não normalizado dos redshifts (coluna 'zphot')
sns.distplot(df['zphot'], kde=False)
plt.show()

In [ ]:
# Histograma normalizado com suavização (KDE) dos redshifts (coluna 'zphot')
sns.distplot(df['zphot'], kde=True)
plt.show()

In [ ]:
# Diagrama _box-and-whisker_ 
sns.boxplot(data=df['zphot'], orient='h')
plt.show()

Vamos definir um novo dataframe apenas com as magnitudes absolutas

In [ ]:
mag_df = pd.DataFrame({'u' : df['mag_u'],
                       'r' : df['mag_r']})

In [ ]:
#plt.figure(dpi=150)
sns.kdeplot(mag_df['u'], shade=True)
sns.kdeplot(mag_df['r'], shade=True)
plt.legend(fontsize=12, loc='upper left')
plt.xlabel('magnitude absoluta')
plt.ylabel('KDE plot')
plt.yticks([])
plt.tight_layout()

In [ ]:
sns.boxplot(data=mag_df, orient='h')
plt.xlabel('magnitude absoluta')
plt.tight_layout()

Calculando a cor u-r

In [ ]:
ur = mag_df['u'] - mag_df['r']
color_df = pd.DataFrame({'u-r':ur})

In [ ]:
color_df.head()

In [ ]:
color_df.describe()

In [ ]:
sns.kdeplot(color_df['u-r'], shade=True, legend='u=r')
plt.legend()#fontsize=12, loc='upper left')
plt.xlabel('color')
plt.ylabel('KDE plot')
plt.yticks([])
plt.tight_layout()

In [ ]:
sns.boxplot(data=color_df, orient='h')
plt.xlabel('color')
plt.tight_layout()

### Diagrama cor-magnitude

<img src="anexos/cmd_theory.jpg" width="500"/>  

In [ ]:
sns.set(style="white", color_codes=True)
sns.jointplot(x=mag_df['r'], y=color_df['u-r'], kind='kde')

In [ ]:
ax = sns.kdeplot(mag_df['r'], color_df['u-r'],
                  shade=True, shade_lowest=False)
plt.plot([-17,-24],[1.25,1.75], '-g')
plt.xlim(-17,-24)
plt.ylim(0,2.5)
 

In [ ]:
mask_red = (color_df['u-r'] >= ((-0.07143 * mag_df['r']) + 0.03571) )
mask_blue = (color_df['u-r'] <= ((-0.07143 * mag_df['r']) + 0.03571) )

In [ ]:

ax = sns.kdeplot(mag_df['r'][mask_blue], color_df['u-r'][mask_blue],
                 cmap="Blues", shade=True, shade_lowest=False)
ax = sns.kdeplot(mag_df['r'][mask_red], color_df['u-r'][mask_red],
                 cmap="Reds", shade=True, shade_lowest=False)
plt.xlim(-17,-24)
plt.ylim(0,2.5)

### 4.3 Utilizando a biblioteca Bokeh


In [ ]:
from bokeh.plotting import figure, show, output_notebook
output_notebook()

def red_gals():

    p1 = figure(title='Red galaxies',plot_width=500, plot_height=500)#, x_range=(-17,-24), y_range=(0,2.5))  

    p1.scatter(df['ra'][mask_red], df['dec'][mask_red], marker='circle', size=15, fill_color='red')

    p1.grid.visible = False
    p1.xaxis.axis_label = 'R.A.'
    p1.yaxis.axis_label = 'Dec.'
    show(p1)

In [ ]:
red_gals()

<a class="anchor" id="export"></a>
# 5. Exportando o notebook para HTML

Para finalizar o tutorial, vamos exportar esses resultados para um arquivo HTML

`File -> Download as -> HTML(.html) `